In [1]:
import pandas as pd
import numpy as np

In [15]:
import mysql.connector
db_con = mysql.connector.connect(host='localhost',user='root',password='Password@123')
cursor=db_con.cursor()

In [3]:
data = pd.read_csv("/home/user/NUVENTO/file1.csv")
data

,SourceTableName,SourceColumnName,TargetTableName,TargetColumnName
0,PSRW_HealthOfficeVisits,Actions,OfficeVisit,Actions_Health_Visit
1,PSRW_HealthOfficeVisits,Assessment,OfficeVisit,Assessment_Comments
2,PSRW_HealthOfficeVisits,ID,OfficeVisit,Health_Visit_ID
3,PSRW_HealthOfficeVisits,Outcome,OfficeVisit,Screening_Outcome
4,PSRW_HealthOfficeVisits,Outcome_ID,OfficeVisit,NaN
...,...,...,...,...
333,PSRW_SchoolEnrollment,SourceID,StudentDetails,NaN
334,PSRW_SchoolEnrollment,SourceTable,STUDENUdostring,NaN
335,PSRW_SchoolEnrollment,StudentID,StudentProfile,StudentID
336,PSRW_SchoolEnrollment,TermID,TermsDetabli,NaN


### dataframe df1,df2 is created with respective columns

In [62]:
df1=data.drop(['TargetTableName','TargetColumnName'],axis=1)
df2=data.drop(['SourceTableName','SourceColumnName'],axis=1)

In [5]:
df1.head()

,SourceTableName,SourceColumnName
0,PSRW_HealthOfficeVisits,Actions
1,PSRW_HealthOfficeVisits,Assessment
2,PSRW_HealthOfficeVisits,ID
3,PSRW_HealthOfficeVisits,Outcome
4,PSRW_HealthOfficeVisits,Outcome_ID


### preparing the list of tables to be created in Database 2

In [6]:
lst=df1.SourceTableName.unique().tolist()
print(lst)

['PSRW_HealthOfficeVisits', 'PS_Enrollment_Al', 'PSRW_HealthGradeLevel', 'PSRW_HealthImmunization', 'PSRW_HealthImmunizationDoses', 'PSRW_HealthMedical', 'PSRW_HealthScreeningTypes', 'PSRW_HealthScreeningWaiver', 'PSRW_HealthStudHearing', 'PSRW_HealthStudOral', 'PSRW_HealthStudScoliosis', 'PSRW_HealthStudTuberculosis', 'PSRW_HealthStudVision', 'PSRW_HealthVitalSigns', 'VWStudentCoreFieldsUC', 'DistrictSchoolView', 'PSSIS_Student_Demographics', 'PSSIS_Student_Enrollments', 'PSRW_SchoolEnrollment']


### Creating tables in the Database 1

In [40]:
for tblName in lst:  
    rslt_df = df1[df1["SourceTableName"] == tblName] 
    lt_col=rslt_df["SourceColumnName"].tolist()
    #print(lt_col)
    SqlQrt="CREATE TABLE " +  tblName + " ("
    colQuery=""
    Col_count=0
    for colName in lt_col:
        Col_count=Col_count+1
        colQuery=colName + " varchar(40) "
        if(len(lt_col)!=Col_count):
            colQuery=colQuery+"," 
        SqlQrt=SqlQrt+" "+ colQuery 
    SqlQrt=SqlQrt + ");"
    cursor.execute("use Database1")
    cursor.execute(SqlQrt)

    print(SqlQrt)

CREATE TABLE PSRW_HealthOfficeVisits ( Actions varchar(40) , Assessment varchar(40) , ID varchar(40) , Outcome varchar(40) , Outcome_ID varchar(40) , ParentGuardian_Contact_Ind varchar(40) , Provider_Name varchar(40) , Provider_Type varchar(40) , StudentID varchar(40) , Time_In varchar(40) , Time_Out varchar(40) , Visit_Date varchar(40) , Visit_Reason varchar(40) , Visit_Type varchar(40) , Visit_Type_ID varchar(40) );
CREATE TABLE PS_Enrollment_Al ( ATT_CalcCntPresentAbsent varchar(40) , ATT_IntervalDuration varchar(40) , EntryDate varchar(40) , EntryCode varchar(40) , ExitDate varchar(40) , ExitCode varchar(40) , Dflt_Att_Mode_Code varchar(40) , Dflt_Conversion_Mode_Code varchar(40) , FTEID varchar(40) , Grade_Level varchar(40) , MembershipShare varchar(40) , ProgramID varchar(40) , SchoolID varchar(40) , StudentID varchar(40) , Track varchar(40) , YearID varchar(40) );
CREATE TABLE PSRW_HealthGradeLevel ( Certification_Comment varchar(40) , Certification_Date varchar(40) , Certificat

CREATE TABLE PSSIS_Student_Enrollments ( Name varchar(40) , Student_Number varchar(40) , Current_School_Abbreviation varchar(40) , Grade varchar(40) , Current_Enroll_Status varchar(40) , EnrollmentCode varchar(40) , EnrollmentType varchar(40) , Full_Time_Equivalency varchar(40) , Enrollment_Transfer_Date_Pend varchar(40) , Enrollment_Transfer_Info varchar(40) , Entry_Code varchar(40) , Entry_Reason varchar(40) , Entry_Date varchar(40) , Exit_Code varchar(40) , Exit_Reason varchar(40) , Exit_Comment varchar(40) , Exit_Date varchar(40) , Graduated_Rank varchar(40) , Graduated_SchoolName varchar(40) , Log_Notes varchar(40) , Transfer_Comments varchar(40) , Withdrawal_Reason varchar(40) , Lunch_Portion_to_Pay varchar(40) , MembershipShare varchar(40) , TuitionPayer varchar(40) , Track varchar(40) );
CREATE TABLE PSRW_SchoolEnrollment ( AsstPrincipal varchar(40) , AsstPrincipalEmail varchar(40) , AsstPrincipalPhone varchar(40) , CountyName varchar(40) , EntryCode varchar(40) , EntryDate var

In [63]:
df2.head()

,TargetTableName,TargetColumnName
0,OfficeVisit,Actions_Health_Visit
1,OfficeVisit,Assessment_Comments
2,OfficeVisit,Health_Visit_ID
3,OfficeVisit,Screening_Outcome
4,OfficeVisit,NaN


### Data Cleaning , preparing the list of tables to be created in Database 2

In [77]:
df2=df2.dropna()
df2Dummy=df2.drop_duplicates()
df2Dummy['TargetTableName'] = df2Dummy['TargetTableName'].str.replace('/','')
df2Dummy['TargetColumnName'] = df2Dummy['TargetColumnName'].str.replace('(','')
df2Dummy['TargetColumnName'] = df2Dummy['TargetColumnName'].str.replace(')','')
lt=df2Dummy.TargetTableName.unique().tolist()
#print(lt)

['OfficeVisit', 'Immunization', 'Inventory', 'Waivers', 'Hearing', 'OralDental', 'Scoliosis', 'Tuberclosis', 'Vision', 'VitalSigns', 'StudentContacts', 'SchoolDetails', 'StudentProfile']


<ipython-input-77-2bccb57705a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2Dummy['TargetTableName'] = df2Dummy['TargetTableName'].str.replace('/','')
<ipython-input-77-2bccb57705a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2Dummy['TargetColumnName'] = df2Dummy['TargetColumnName'].str.replace('(','')
<ipython-input-77-2bccb57705a4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

### Creating tables in the Database 2

In [78]:
for tblName in lt:  
    rslt_df = df2Dummy[df2Dummy["TargetTableName"] == tblName] 
    lt_col=rslt_df["TargetColumnName"].tolist()
    #print(lt_col)
    SqlQrt="CREATE TABLE " +  tblName + " ("
    colQuery=""
    Col_count=0
    for colName in lt_col:
        Col_count=Col_count+1
        colQuery=colName + " varchar(40) "
        if(len(lt_col)!=Col_count):
            colQuery=colQuery+"," 
        SqlQrt=SqlQrt+" "+ colQuery 
    SqlQrt=SqlQrt + ");"
    cursor.execute("use Database2")
    cursor.execute(SqlQrt)

   # print(SqlQrt)

CREATE TABLE OfficeVisit ( Actions_Health_Visit varchar(40) , Assessment_Comments varchar(40) , Health_Visit_ID varchar(40) , Screening_Outcome varchar(40) , ParentGuardian_Contact_Ind varchar(40) , Provider_Name varchar(40) , Provider_Type varchar(40) , StudentID varchar(40) , Time_In varchar(40) , Time_Out varchar(40) , Visit_Date varchar(40) , Visit_Reason varchar(40) , Visit_Type varchar(40) , Visit_Type_ID varchar(40) );
CREATE TABLE Immunization ( Administered_Dose_Count varchar(40) , Compliance_Status varchar(40) , Exemption_Code varchar(40) , Exemption_Reason varchar(40) , Exemption_State_Code varchar(40) , Exemption_State_Reason varchar(40) , Exemption_Status varchar(40) , Immunization_Comment varchar(40) , StudentID varchar(40) , Vaccine_Code varchar(40) , Vaccine_Compliance_Mode varchar(40) , Vaccine_Description varchar(40) , Vaccine_Dose_Count varchar(40) , Vaccine_ID varchar(40) , Vaccine_MandatoryOptional varchar(40) , Vaccine_Name varchar(40) , Vaccine_Sort_Order varchar

In [101]:
for tblName in lt:  
    rslt_df = df2Dummy[df2Dummy["TargetTableName"] == tblName] 
    lt1_col=rslt_df["TargetColumnName"].tolist()
  
    SqlQrt="INSERT INTO " + tblName + " ("
    colQuery=""
    Col_cout=0
    for colName in lt1_col:
        Col_count=Col_cout+1
        colQuery=colName
        if(len(lt1_col)!=Col_cout):
            colQuery=colQuery+","
        
        SqlQrt=SqlQrt+colQuery 
    SqlQrt=SqlQrt + ")"
    
    valueQuery=" VALUES ("
    valueCount=0
    for colName in lt1_col:
        valueCount=valueCount+1
        if(len(lt1_col)!=valueCount):
            valueQuery= valueQuery +"Dummy Text"+"," 
        else:
             valueQuery= valueQuery +"Dummy Text"
        
    valueQuery=valueQuery + ")"
    
    SqlQrt=SqlQrt+ valueQuery
#     cursor.execute("use Database2")
#     cursor.execute(SqlQrt)

    print(SqlQrt)

INSERT INTO OfficeVisit (Actions_Health_Visit,Assessment_Comments,Health_Visit_ID,Screening_Outcome,ParentGuardian_Contact_Ind,Provider_Name,Provider_Type,StudentID,Time_In,Time_Out,Visit_Date,Visit_Reason,Visit_Type,Visit_Type_ID,) VALUES (Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text,Dummy Text)
INSERT INTO Immunization (Administered_Dose_Count,Compliance_Status,Exemption_Code,Exemption_Reason,Exemption_State_Code,Exemption_State_Reason,Exemption_Status,Immunization_Comment,StudentID,Vaccine_Code,Vaccine_Compliance_Mode,Vaccine_Description,Vaccine_Dose_Count,Vaccine_ID,Vaccine_MandatoryOptional,Vaccine_Name,Vaccine_Sort_Order,Vaccine_State_Code,Vaccine_State_Name,Certificate_Type_Code,Certificate_Type_Name,Certificate_Type_State_Code,Certificate_Type_State_Name,Compliancy_Statement,Compliancy_Status,Dose_Date_Administered,Dose_Date_Recorded,Dose_Number,Is_Default_Certificate_Type,Vaccinat